In [1]:
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import pickle

# Import the model & train/test files

In [2]:
model = joblib.load('gbr_x3_model.pkl')
train_X3 = pd.read_csv('X_train_3.csv')
test_X3 = pd.read_csv('X_test_3.csv')
train_y3 = pd.read_csv('y_train_3.csv')
test_y3 = pd.read_csv('y_test_3.csv')

All of the imported DFs have the mysterious 'Unnamed: 0' column. Let's drop that from each of the DFs.

In [3]:
train_X3 = train_X3.drop(columns='Unnamed: 0')
test_X3 = test_X3.drop(columns='Unnamed: 0')
train_y3 = train_y3.drop(columns='Unnamed: 0')
test_y3 = test_y3.drop(columns='Unnamed: 0')

The y frames need to be changed to 1D arrays for GridSearch to work correctly.

In [4]:
train_y3 = train_y3.values.ravel()
test_y3 = test_y3.values.ravel()

# Tune the Hyperparameters

In [5]:
#Define dictionary with possible parameters
params = {
    'learning_rate': [0.1, 0.05, 0.01],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [1.0, 'sqrt', 'log2']}

#Perform GridSearch
grid_search = GridSearchCV(
    estimator=model,
    param_grid=params,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    cv=5,
    verbose=1
)

grid_search.fit(train_X3, train_y3)

print("Best Parameters:", grid_search.best_params_)
print("Best MSE Score:", grid_search.best_score_)

best_model = grid_search.best_estimator_


Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Best MSE Score: -1.0416675882872979


Since I specified neg_mean_squared_error as the scoring parameter, this MSE is actually 1.1252. That score is actually higher than the non-optimized value from the previous notebook (1.029). That being the case, I will perform another round of optimization to try to improve the MSE.

In [6]:
#Define dictionary with possible parameters
#n_estimators was found to be optimal at the default value, so it will be omitted below
#since default values perform better, this search will allow max_features to default
params_2 = {
    'learning_rate': [0.01, 0.005, 0.0001],
    'max_depth': [4, 5, 6],
    'min_samples_split': [10, 25, 50],
    'min_samples_leaf': [4, 10, 25]}

#Perform GridSearch
grid_search_2 = GridSearchCV(
    estimator=model,
    param_grid=params_2,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    cv=5,
    verbose=1)

grid_search_2.fit(train_X3, train_y3)

print("Best Parameters:", grid_search_2.best_params_)
print("Best MSE Score:", grid_search_2.best_score_)

best_model_2 = grid_search_2.best_estimator_

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Parameters: {'learning_rate': 0.01, 'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 10}
Best MSE Score: -1.1268130595402268


The MSE is still not better than the default value, but it's closer. I will tune hyperparameters once more.

In [7]:
#Define dictionary with possible parameters
#n_estimators and max_features are defaulted
#learning_rate and min_samples_leaf were the same across both tunings, so we will specify those values but not further tune them

params_3 = {
    'learning_rate': [0.01],
    'max_depth': [1, 2, 3, 4],
    'min_samples_leaf': [4],
    'min_samples_split': [50, 75, 100]}

#Perform GridSearch
grid_search_3 = GridSearchCV(
    estimator=model,
    param_grid=params_3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    cv=5,
    verbose=1)

grid_search_3.fit(train_X3, train_y3)

print("Best Parameters:", grid_search_3.best_params_)
print("Best MSE Score:", grid_search_3.best_score_)

best_model_3 = grid_search_3.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'learning_rate': 0.01, 'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 50}
Best MSE Score: -1.1511442926262707


This tuning resulted in the same set of hyperparameters for the model as the previous round. Hyperparameter tuning is complete.

The MSE is still not as low as the generic model, but it is quite close, so I will keep it to make predictions below.b

In [8]:
#Update the model with the optimized hyperparameters
hyperparameters = {
    'learning_rate': 0.01,
    'max_depth': 4,
    'min_samples_leaf': 4,
    'min_samples_split': 50}

#Update the hyperparameters
model.set_params(**hyperparameters)

#Fit the model to the training data to prepare for Gaussian Processes
model.fit(train_X3, train_y3)

GradientBoostingRegressor(learning_rate=0.01, max_depth=4, min_samples_leaf=4,
                          min_samples_split=50, random_state=42)

## Make Predictions with the Model

In [9]:
#Make predictions on training and test set
train_preds = model.predict(train_X3)
test_preds = model.predict(test_X3)

# Gaussian Processes

In [10]:
#Calculate residuals
train_residuals = train_y3 - train_preds

#Define the kernel for GPR
kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0,(1e-4, 1e1))

#Train the GPR model on residuals
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-2)
gpr.fit(train_X3, train_residuals)

#Make predictions on test set with GPR
gpr_mean, gpr_std = gpr.predict(test_X3, return_std=True)

#Combine GBR with GPR uncertainty
final_preds = test_preds + gpr_mean

#Calculate 95% confidence intervals
#95% of data falls within 1.96 SD of the mean, so that standard is used to calculate confidence intervals
lower_bound = final_preds - 1.96 * gpr_std
upper_bound = final_preds + 1.96 * gpr_std

#Print the results
for i, (pred, lower, upper) in enumerate(zip(final_preds, lower_bound, upper_bound)):
    print(f"Prediction: {pred:.4f}, Lower Bound: {lower:.4f}, Upper Bound: {upper:.4f}")

Prediction: 16.7918, Lower Bound: 14.7501, Upper Bound: 18.8334
Prediction: 16.8315, Lower Bound: 14.7899, Upper Bound: 18.8732
Prediction: 17.0356, Lower Bound: 14.9939, Upper Bound: 19.0773
Prediction: 16.9449, Lower Bound: 14.9032, Upper Bound: 18.9866
Prediction: 16.9449, Lower Bound: 14.9032, Upper Bound: 18.9866
Prediction: 16.9449, Lower Bound: 14.9032, Upper Bound: 18.9866
Prediction: 17.1914, Lower Bound: 15.1497, Upper Bound: 19.2331
Prediction: 16.7490, Lower Bound: 14.7073, Upper Bound: 18.7907
Prediction: 16.7780, Lower Bound: 14.7364, Upper Bound: 18.8197
Prediction: 16.6711, Lower Bound: 14.6295, Upper Bound: 18.7128
Prediction: 16.9490, Lower Bound: 14.9074, Upper Bound: 18.9907
Prediction: 16.9984, Lower Bound: 14.9567, Upper Bound: 19.0400
Prediction: 16.9230, Lower Bound: 14.8813, Upper Bound: 18.9647
Prediction: 16.9771, Lower Bound: 14.9355, Upper Bound: 19.0188
Prediction: 16.9230, Lower Bound: 14.8813, Upper Bound: 18.9647
Prediction: 17.0179, Lower Bound: 14.976

# Export the models

Since we are utilizing Gaussian Processes, we have to export both the Gradient Boosting Regressor model and the Gaussian Process model for deployment in the User Interface.

In [11]:
with open('gbr_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('gpr_model.pkl', 'wb') as f:
    pickle.dump(gpr, f)

In [14]:
import os

print(os.listdir(os.getcwd()))

['cleaned_data', 'feature_3.csv', 'X_train_3.csv', 'X_test_3.csv', '.DS_Store', 'merged_dataset', '1-- Import and Clean Initial Data-checkpoint.ipynb', 'weather_data.csv', 'gbr_x3_model.pkl', '3-- Descriptive Statistics and Visualizations-checkpoint.ipynb', 'gpr_model.pkl', 'Production.csv', '4 -- Feature Selection & Preprocessing-checkpoint.ipynb', 'feature_10.csv', '6-- Hyperparamter Tuning & Gaussian Processes.ipynb', 'static', 'preprocessing_pipeline.pkl', 'y_train_3.csv', 'y_test_3.csv', 'templates', '.ipynb_checkpoints', 'Production.numbers', '5 -- Initial Model Training and Evaluation.ipynb', 'web_model.py', 'myenv', 'gbr_model.pkl', '2-- Add Weather Data-checkpoint.ipynb']
